# Firm-years with less than 200 trading days
author: Robert Bulava, https://github.com/robert-bulava , copyright: MIT License

<h1>Table of Contents<span class="tocSkip"></span></h1>
<div class="toc"><ul class="toc-item"><li><span><a href="#Firm-years-with-less-than-200-trading-days" data-toc-modified-id="Firm-years-with-less-than-200-trading-days-1"><span class="toc-item-num">1&nbsp;&nbsp;</span>Firm-years with less than 200 trading days</a></span><ul class="toc-item"><li><span><a href="#Setting-the-environment" data-toc-modified-id="Setting-the-environment-1.1"><span class="toc-item-num">1.1&nbsp;&nbsp;</span>Setting the environment</a></span></li><li><span><a href="#Data-loading-and-inspection" data-toc-modified-id="Data-loading-and-inspection-1.2"><span class="toc-item-num">1.2&nbsp;&nbsp;</span>Data loading and inspection</a></span></li><li><span><a href="#Variable-calculation" data-toc-modified-id="Variable-calculation-1.3"><span class="toc-item-num">1.3&nbsp;&nbsp;</span>Variable calculation</a></span><ul class="toc-item"><li><span><a href="#Active-trading-days" data-toc-modified-id="Active-trading-days-1.3.1"><span class="toc-item-num">1.3.1&nbsp;&nbsp;</span>Active trading days</a></span></li><li><span><a href="#Active-trading-years" data-toc-modified-id="Active-trading-years-1.3.2"><span class="toc-item-num">1.3.2&nbsp;&nbsp;</span>Active trading years</a></span></li></ul></li><li><span><a href="#Export" data-toc-modified-id="Export-1.4"><span class="toc-item-num">1.4&nbsp;&nbsp;</span>Export</a></span></li></ul></li></ul></div>

__Implemented text:__ <br> 
_'...To ensure there are enough data points to compute liquidity measures, we exclude firm-year observations with lower than two hundred active trading days in a year...' (Brogaard et al., 2017)_ (Section 2. Data and variable construction, paragraph 1)

__Data source:__ Compustat - Capital IQ through Wharton Research Data Services (https://wrds-www.wharton.upenn.edu/)

__Dataset:__ Compustat Daily Updates - Security Daily (1993-01-01 -- 2013-12-31)

__Action:__ Filter out firm-years with less than 200 trading days. 

__Script Output:__ A data frame with columns _gvkey, year, actively_traded_ (days in a year), and <i>to_keep</i> (True or False). This data frame can be merged with a Compustat Industrial Files data frame in order to determine which firm-years need to be removed. 

## Setting the environment

In [1]:
import os
import pandas as pd
import numpy as np
from IPython.display import Image

In [2]:
pd.set_option('display.max_columns', None)
pd.set_option('display.max_rows', 150)
pd.set_option('display.expand_frame_repr', False)
pd.set_option('max_colwidth', None)

In [3]:
pd.set_option("mode.chained_assignment", "raise")

In [4]:
root_folder = r"C:\Pandas\BrogaardEtAlReplication\Table_4"
os.chdir(root_folder)

## Data loading and inspection

__Variables:__ _gvkey, datadate, cshtrd_ (Trading Volume - Daily) _, tpci_ (Issue Type Code), _exch_ (Stock Exchange Code).

In [5]:
usecols = ['gvkey', 'datadate', 'cshtrd', 'tpci', 'exchg']
dtype = {'gvkey': 'uint32', 'datadate': 'int32',
         'cshtrd': 'float32', 'tpci': 'object', 'exchg': 'Int8'}

Time period is 1993-01-01 -- 2013-12-31.

In [6]:
time_span = np.arange(1993, 2014)  # as specified in the paper

Issue Type Code - only common stocks will be kept in the data frame.

Stock Exchange code (<i>exch</i>) - only US stock exchanges will be included.

In [7]:
us_exchanges = list(range(2, 5)) + list(range(11, 21))

In [8]:
cdaily = pd.DataFrame()

chunksize = 10**6

for chunk in pd.read_csv(r"data\cdaily_enriched2.csv", usecols=usecols, dtype=dtype, chunksize=chunksize):

    # removing rows with stocks that are not common or traded on non-US exchanges
    chunk = chunk.loc[(chunk['tpci'] == '0') & (
        chunk['exchg'].isin(us_exchanges)), ['gvkey', 'datadate', 'cshtrd']]

    # processing datadate
    chunk['datadate'] = pd.to_datetime(chunk["datadate"], format='%Y%m%d')
    chunk['year'] = chunk['datadate'].dt.year.astype('uint16')

    # keeping rows within given time period
    chunk = chunk.loc[chunk['year'].isin(time_span), ]

    cdaily = pd.concat([cdaily, chunk], ignore_index=True, copy=True)

In [9]:
cdaily.shape

(54615547, 4)

Percentage of NaN in columns.

In [10]:
(cdaily.isna().sum()/len(cdaily)*100).round(4).astype(str) + ' %'

gvkey          0.0 %
datadate       0.0 %
cshtrd      0.0246 %
year           0.0 %
dtype: object

## Variable calculation

### Active trading days

_'...To ensure there are enough data points to compute liquidity measures, we exclude firm-year observations with lower than two hundred active trading days in a year...' (Brogaard et al., 2017)_

Some stocks are traded on multiple exchanges. I will assume that a stock is actively traded in a day if it is traded on one or more stock exchanges. In other words, the sum of its trade volumes is greater than zero across all the relevant exchanges.

In [11]:
cdaily = cdaily.groupby(['gvkey', 'datadate'])['cshtrd'].sum().reset_index()

In [12]:
cdaily.shape

(54070921, 3)

In [13]:
cdaily['actively_traded'] = np.where(cdaily['cshtrd'] > 0, True, False)

In [14]:
cdaily.shape

(54070921, 4)

In [15]:
cdaily = cdaily.loc[cdaily['actively_traded'] ==
                    True, ['gvkey', 'datadate', 'actively_traded']]

In [16]:
cdaily.shape

(39526967, 3)

### Active trading years
i.e. with 200 or more non-zero trades in a year.

In [17]:
cdaily['year'] = cdaily['datadate'].dt.year.astype('uint16')

In [18]:
cdaily.shape

(39526967, 4)

In [19]:
cdaily = cdaily.groupby(['gvkey', 'year'])[
    'actively_traded'].sum().astype('uint16').reset_index()

In [20]:
cdaily['to_keep'] = np.where(cdaily['actively_traded'] >= 200, True, False)

In [21]:
cdaily.sample(10)

,gvkey,year,actively_traded,to_keep
188286,137580,2008,11,False
54864,13113,1996,254,True
73771,17795,2000,130,False
92511,24840,2013,7,False
84987,23062,1997,253,True
50661,12445,2007,251,True
83261,22461,1997,253,True
74716,18332,2010,147,False
42498,10733,1999,223,True
152151,65064,1999,184,False


In [22]:
# check, there shouldn't be more than 255 trading days in a year
cdaily.loc[cdaily['actively_traded'] > 255, ]

,gvkey,year,actively_traded,to_keep


In [23]:
cdaily.shape

(227706, 4)

## Export

In [24]:
cdaily.to_csv(r'data/active_firm_years.csv',
              columns=['gvkey', 'year', 'actively_traded', 'to_keep'])